https://stackoverflow.com/questions/73442770/implement-bin-packing-problem-with-additional-elastic-constraints-using-pulp-opt

In [1]:
import pulp
from itertools import product
import pandas as pd
import numpy as np

# DataFrame of item, weight, and length
df_updated = pd.DataFrame(
    [
        ["item1", 10, "A"],
        ["item2", 20, "B"],
        ["item3", 20, "C"],
        ["item4", 20, "B"],
        ["item5", 10, "A"],
        ["item6", 10, "B"],
    ],
    columns=["itemname", "QuantityToGroup", "Length"],
)
lengths = list(df_updated.Length.unique())

# Max weightage per bin
max_weight = 40

# big M for number of items
big_M = len(df_updated)

# Max bin to use
min_bins = int(np.ceil(round((df_updated["QuantityToGroup"].sum() / max_weight))))
max_bins = 3
bins = list(range(max_bins))

problem = pulp.LpProblem("Grouping_lengths", pulp.LpMinimize)

# Variable to check, if we are using the bin or not
bin_used = pulp.LpVariable.dicts(
    "is_bin_used", bins, cat="Binary"
)

# Indicator that items of dimension d are located in bin b:
loaded = pulp.LpVariable.dicts(
    "loaded", [(d, b) for d in lengths for b in bins], cat="Binary"
)

# the total count of bins used
tot_bins = pulp.LpVariable("bins_used")

# the total count of overloads in a bin.  overload = (count of dimensions in bin) - 1
overload = pulp.LpVariable.dicts("bin_overloads", bins, lowBound=0)

# Possible combinations to put the item in the bin
possible_item_in_bin = [
    (item_index, bin_num) for item_index, bin_num in product(df_updated.index, bins)
]
item_in_bin = pulp.LpVariable.dicts(
    "is_item_in_bin", possible_item_in_bin, cat="Binary"
)

# Force each item to be loaded...
for item_index in df_updated.index:
    problem += (
        pulp.lpSum([item_in_bin[item_index, bin_index] for bin_index in bins]) == 1,
        f"Ensure that item {item_index} is only in one bin",
    )

# Sum of quantity grouped in each bin must be less than max weight
for bin_index in bins:
    problem += (
        pulp.lpSum(
            [
                item_in_bin[item_index, bin_index]
                * df_updated.loc[item_index, "QuantityToGroup"]
                for item_index in df_updated.index
            ]
        )
        <= max_weight * bin_used[bin_index],
        f"Sum of items in bin {bin_index} should not exceed max weight {max_weight}",
    )

# count the number of dimensions (lengths) in each bin
for b in bins:
    for d in lengths:
        problem += loaded[d, b] * big_M >= pulp.lpSum(
            item_in_bin[idx, b] for idx in df_updated.index[df_updated.Length == d]
        )

# attach the "bin used" variable to either the "loaded" var or "item in bin" var...
for b in bins:
    problem += bin_used[b] * big_M >= pulp.lpSum(
        item_in_bin[idx, b] for idx in df_updated.index
    )

# count total bins used
problem += tot_bins >= pulp.lpSum(bin_used[b] for b in bins)

# count the overloads by bin
for b in bins:
    problem += overload[b] >= pulp.lpSum(loaded[d, b] for d in lengths) - 1


# Objective function to minimize bins used, with some small penalty for total overloads
usage_wt = 1.0
overload_wt = 0.2

problem += (
    usage_wt * tot_bins + overload_wt * pulp.lpSum(overload[b] for b in bins),
    "Objective: Minimize Bins Used, penalize overloads",
)

problem.solve(pulp.PULP_CBC_CMD(msg=False))
status = pulp.LpStatus[problem.status]
assert(status=='Optimal')  # <--- always ensure this before looking at result if you don't print solve status


print(f"total bins used: {tot_bins.varValue}")
print("bin overloads:")
for b in bins:
    if overload[b].varValue > 0:
        print(f"    bin {b} has {overload[b].varValue} overloads")

for idx, b in possible_item_in_bin:
    if item_in_bin[idx, b].varValue == 1:
        print(
            f"load {df_updated.itemname.iloc[idx]}/{df_updated.Length.iloc[idx]} in bin {b}"
        )

total bins used: 3.0
bin overloads:
    bin 0 has 1.0 overloads
load item1/A in bin 0
load item2/B in bin 2
load item3/C in bin 1
load item4/B in bin 2
load item5/A in bin 0
load item6/B in bin 0
